# install libraries

In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 fosforio fosforml numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15

# Import helper libraries

In [1]:
from fosforio import snowflake
from fosforml import *
from fosforml.constants import MLModelFlavours
from fosforio import get_dataframe
from matplotlib import pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
import warnings; warnings.simplefilter('ignore')
from joblib import dump, load
import requests
from tqdm import tqdm
import time
import calendar

from time import sleep
import configparser
from dateutil.relativedelta import relativedelta
import datetime
from dateutil.easter import easter
from scipy.optimize import minimize_scalar
from scipy.optimize import curve_fit

%matplotlib inline

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


/tmp/pip_packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Matplotlib created a temporary cache directory at /tmp/matplotlib-x3_had4z because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# connect to snowflake

In [4]:
snowflake.get_connection(connection_name="TTH_REV_OPT_CXN")
data = get_dataframe("BOOKINGS_TRANSFORMED")
data

Connection object created: <snowflake.connector.connection.SnowflakeConnection object at 0x7f40f2b8e250>
Please close the connection after use!
Reading dataframe from snowflake native connector


,ARRIVAL_DATE_TRANSFORMED,RESERVATION_STATUS_DATE_TRANSFORMED,HOTEL,IS_CANCELED,LEAD_TIME,ARRIVAL_DATE_YEAR,MONTH,ARRIVAL_DATE_WEEK_NUMBER,ARRIVAL_DATE_DAY_OF_MONTH,EXPECTED_ARRIVAL_DATE,RESERVATION_STATUS,RESERVATION_STATUS_DATE,TOTAL_STAY_NIGHTS,TALLY_DAYS,STAYS_IN_WEEKEND_NIGHTS,STAYS_IN_WEEK_NIGHTS,ADULTS,CHILDREN,BABIES,TOTAL_GUESTS,AVG_ROOMS_PER_NIGHT,TOTAL_ROOM_NIGHTS,MEAL,COUNTRY,MARKET_SEGMENT,DISTRIBUTION_CHANNEL,PREVIOUS_CANCELLATIONS,PREVIOUS_BOOKINGS_NOT_CANCELED,RESERVED_ROOM_TYPE,ASSIGNED_ROOM_TYPE,DEPOSIT_TYPE,DAYS_IN_WAITING_LIST,CUSTOMER_TYPE,ADR
0,2021-07-16,2020-10-17,City Hotel,1,272,2021,7,29,16,16-07-2021,Canceled,17-10-2020,2,0.272,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
1,2021-08-20,2020-10-17,City Hotel,1,307,2021,8,34,20,20-08-2021,Canceled,17-10-2020,2,0.307,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
2,2021-07-02,2020-10-17,City Hotel,1,258,2021,7,27,2,02-07-2021,Canceled,17-10-2020,2,0.258,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
3,2021-08-20,2020-10-17,City Hotel,1,307,2021,8,34,20,20-08-2021,Canceled,17-10-2020,2,0.307,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
4,2021-07-23,2020-10-17,City Hotel,1,279,2021,7,30,23,23-07-2021,Canceled,17-10-2020,2,0.279,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109706,2023-08-24,2023-09-10,Resort Hotel,0,269,2023,8,34,24,24-08-2023,Check.Out,10-09-2023,17,17.000,4,13,2,0,0,2,1,17,BB,GBR,Offline TA/TO,TA/TO,0,0,D,D,No Deposit,0,Contract,84.80
109707,2023-08-31,2023-09-10,Resort Hotel,0,212,2023,8,35,31,31-08-2023,Check.Out,10-09-2023,10,10.000,2,8,2,1,0,3,1,10,BB,GBR,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,89.75
109708,2023-08-29,2023-09-12,Resort Hotel,0,204,2023,8,35,29,29-08-2023,Check.Out,12-09-2023,14,14.000,4,10,2,0,0,2,1,14,BB,IRL,Direct,Direct,0,0,E,E,No Deposit,0,Transient,153.57
109709,2023-08-31,2023-09-14,Resort Hotel,0,161,2023,8,35,31,31-08-2023,Check.Out,14-09-2023,14,14.000,4,10,2,0,0,2,1,14,HB,DEU,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,99.06


In [7]:
import pandas as pd
import numpy as np
from faker import Faker

# Initialize Faker to generate random data
fake = Faker()

# Number of rows
num_rows = 109711

# Generate the B_ID column
# Function to generate a realistic transaction ID
def generate_transaction_id(i):
    prefix = 'B'
    date_component = data['ARRIVAL_DATE_TRANSFORMED'].iloc[i].strftime("%Y%m%d")
    unique_component = str(fake.random_number(digits=6, fix_len=True))
    return f"{prefix}{date_component}{unique_component}"

# Generate the B_ID column
b_ids = [generate_transaction_id(i) for i in range(num_rows)]

# Generate random dates
dates = [fake.date_this_decade() for _ in range(num_rows)]

# Generate random amounts
amounts = np.round(np.random.uniform(1.0, 1000.0, num_rows), 2)

# Generate transaction types
transaction_types = np.random.choice(['Credit', 'Debit'], num_rows)

# Generate descriptions
descriptions = [fake.sentence(nb_words=5) for _ in range(num_rows)]

# Create the DataFrame
datas = {
    'B_ID': b_ids,
    'Date': dates,
    'Amount': amounts,
    'Type': transaction_types,
    'Description': descriptions
}

df = pd.DataFrame(datas)

df

,B_ID,Date,Amount,Type,Description
0,B20210716694520,2021-05-17,986.57,Debit,Item sit from wide food.
1,B20210820815439,2022-07-09,704.14,Credit,Phone cultural west wish.
2,B20210702111301,2024-06-24,208.23,Debit,Modern size special professional who.
3,B20210820666596,2020-02-26,978.97,Debit,Too trouble letter.
4,B20210723840280,2020-09-11,683.54,Debit,Media he half career test beyond.
...,...,...,...,...,...
109706,B20230824681605,2020-01-29,358.35,Credit,And other follow rise federal.
109707,B20230831460007,2023-01-16,526.92,Debit,For rest support detail.
109708,B20230829449177,2021-08-20,739.87,Credit,Stay agreement little condition.
109709,B20230831743379,2021-06-19,861.54,Debit,Walk after fear sell but should.


In [ ]:
data['Transcation_IDs'] = df['B_ID']

In [ ]:
cust = get_dataframe("CUSTOMER_DATA")
cust

datetime.date(2021, 8, 20)

In [ ]:
data.columns = [x.lower() for x in data.columns]

In [ ]:
data.columns

# removing Canceletions and no-shows and keep City hotel data only


In [ ]:
df = data[(data['is_canceled'] == 0) & (data['reservation_status'] !='No-Show')] 
df

In [ ]:
df.groupby(['hotel','market_segment','reserved_room_type']).agg({'adr':'mean','reservation_status_date_transformed':'count'})


In [ ]:
data = df[(df.market_segment != 'Complementary') ]
data = data[(data.reserved_room_type == 'A') |(data.reserved_room_type == 'D') | (data.reserved_room_type == 'E')]
data.reserved_room_type.value_counts()


In [ ]:
def update_values(row):
    if row['hotel'] == 'City Hotel' and row['reserved_room_type'] == 'A':
        return 460
    elif row['hotel'] == 'City Hotel' and row['reserved_room_type'] == 'D':
        return 135
    elif row['hotel'] == 'City Hotel' and row['reserved_room_type'] == 'E':
        return 30
    elif row['hotel'] == 'Resort Hotel' and row['reserved_room_type'] == 'A':
        return 360
    elif row['hotel'] == 'Resort Hotel' and row['reserved_room_type'] == 'D':
        return 125
    elif row['hotel'] == 'Resort Hotel' and row['reserved_room_type'] == 'E':
        return 100
    else:
        pass

In [ ]:
data['room_limit'] = data.apply(update_values, axis=1)
data

In [ ]:
data.info()


In [ ]:
data_backup = data.copy()

In [ ]:
hotels = data['hotel'].unique()
room_types = data['reserved_room_type'].unique()

In [ ]:
data['total_rns'] = data['stays_in_week_nights'] + data['stays_in_weekend_nights']
data_to_transform = data[['hotel','reserved_room_type','arrival_date_transformed','total_rns','adr', 'room_limit']]
data_to_transform

In [ ]:
import os
from snowflake.snowpark.session import Session
user = os.getenv("user")
warehouse = os.getenv("warehouse")
schema= os.getenv("schema")
database = os.getenv("database")
role =  os.getenv("role")
account =  os.getenv("account")
password= os.getenv("password")

connection_params = dict(user=user, 
                         password=password, 
                         account=account, 
                         warehouse=warehouse, 
                         database=database,
                         schema=schema, 
                         role=role)

session = Session.builder.configs(connection_params).create()

session.sql('use warehouse {};'.format(warehouse)).collect()

session.sql('use database {};'.format(database)).collect()

session.sql('use schema {}.{};'.format(database, schema)).collect()

# Create a new dataframe to store the data by stay date

In [ ]:
expanded_df = pd.DataFrame()

for _, row in data.iterrows():
    num_stay_dates = row['total_rns']
    try:
        # Create a row for each stay date
        expanded_booking = pd.DataFrame({
            'hotel': row['hotel'],
            'room_type': row['reserved_room_type'], 
            'arrival_date': pd.date_range(start=row['expected_arrival_date'], periods=num_stay_dates),
            'total_rns': 1,
            'adr': row['adr'],
            'room_limit': row['room_limit']
        })
        
        # Append the stay date information to the new dataframe
        expanded_df = pd.concat([expanded_df, expanded_booking], ignore_index=True)
    except ValueError as e:
        print(f"Error processing booking for {row['hotel']} on {row['expected_arrival_date']} : {num_stay_dates} {e}")

# Sort the final dataframe by date

In [ ]:
expanded_df = expanded_df.sort_values('arrival_date')
expanded_df = expanded_df.reset_index(drop=True)
expanded_df

In [ ]:
expanded_df['adr']= np.round(expanded_df['adr'], 2)

expanded_df


# Building seasonality

In [ ]:
import holidays
holiday_dates = holidays.CountryHoliday('PT', years=[2020,2021,2022,2023])
holidays = {
    expected_arrival_date: name
    for expected_arrival_date, name in holiday_dates.items()
    if name in ['Ano Novo', 'Páscoa', 'Dia de Natal']
}

# rename holiday columns

In [ ]:
expanded_df = expanded_df.rename({'Ano Novo':'new_year','Páscoa':'easter','Dia de Natal':'christmas'},axis=1)

In [ ]:
def generate_holiday_dates(start_year, end_year):
    holidays = {}
    for year in range(start_year, end_year + 1):
        holidays[datetime.date(year, 1, 1)] = 'new_year'
        easter_date = easter(year)
        holidays[easter_date] = 'easter'
        holidays[datetime.date(year, 12, 25)] = 'christmas'
    return holidays

holidays = generate_holiday_dates(2020, 2023)


In [ ]:
holidays

In [ ]:
# Define pre and post ranges for each holiday
pre_range_offset = {'new_year': relativedelta(days=-1),
                    'easter': relativedelta(days=-2),
                    'christmas': relativedelta(days=-3)}

post_range_offset = {'new_year': relativedelta(days=1),
                     'easter': relativedelta(days=2),
                     'christmas': relativedelta(days=3)}

In [ ]:
# Create new columns for each holiday
for holiday in holidays.values():
    expanded_df[holiday] = 0
 
 # Set the holiday columns to 1 for matching dates
for arrival_date, name in holidays.items():
    expanded_df.loc[expanded_df['arrival_date'] == arrival_date, name] = 1

    # Set the holiday columns to 1 for pre and post dates
    pre_offset = pre_range_offset.get(name)
    if pre_offset:
        pre_date = pd.to_datetime(arrival_date) + pre_offset
        expanded_df.loc[expanded_df['arrival_date'] == pre_date.strftime('%Y-%m-%d'), name] = 1

    post_offset = post_range_offset.get(name)
    if post_offset:
        post_date = pd.to_datetime(arrival_date) + post_offset
        expanded_df.loc[expanded_df['arrival_date'] == post_date.strftime('%Y-%m-%d'), name] = 1

In [ ]:
##check
expanded_df.head()

In [ ]:
# Add dow, month to data
expanded_df['dow'] = expanded_df.arrival_date.dt.strftime('%A')
expanded_df['month'] = expanded_df.arrival_date.dt.strftime('%B')

In [ ]:
##check
expanded_df.head()

In [ ]:
expanded_df.shape

In [ ]:
expanded_df.info()

In [ ]:
expanded_df[expanded_df['easter'] == 0]

In [ ]:
expanded_df['adr'] = np.round(expanded_df['adr'], 2)
expanded_df

# Non holidays dataset

In [ ]:
non_holidays = expanded_df[expanded_df[['new_year', 'easter', 'christmas']].sum(axis=1) == 0]

non_holidays.head()

In [ ]:
non_holidays.shape

In [ ]:
daily_rns= non_holidays.groupby(['arrival_date','dow','month', 'hotel', 'room_type']).agg({'room_limit': 'mean', 'total_rns':'sum'}).reset_index() # ge total stays per day

daily_rns = daily_rns.groupby(['dow','month', 'hotel', 'room_type']).agg({'room_limit': 'mean','total_rns':['sum','mean','median']}).reset_index() # get Rns metrics by Dow & Month

daily_rns.columns = ['_'.join(col) for col in daily_rns.columns] #remove multi level column
daily_rns

In [ ]:
adr_frequency = non_holidays.groupby(['dow','month','adr', 'hotel', 'room_type']).agg({'room_limit': 'mean','total_rns':'sum'})
adr_frequency.reset_index(inplace=True)

In [ ]:
adr_frequency

In [ ]:
merged_df = pd.merge(adr_frequency, daily_rns,how='left',left_on=['dow','month', 'hotel', 'room_type'], right_on=['dow_','month_', 'hotel_', 'room_type_'],suffixes=('_act', '_tot'))

merged_df = merged_df.drop(['dow_','month_'],axis=1)

merged_df

In [ ]:
merged_df['probability'] = merged_df['total_rns']/merged_df['total_rns_sum']

In [ ]:
merged_df['expected_rns'] = merged_df['probability'] * merged_df['total_rns_median']

In [ ]:
merged_df = merged_df.sort_values(by=['dow', 'month', 'adr'], ascending=[True, True, False])

In [ ]:
merged_df['expected_demand']=merged_df.groupby(['dow', 'month'])['expected_rns'].cumsum()

In [ ]:
merged_df['expected_rev'] = merged_df['adr']* merged_df['expected_demand']

In [ ]:
merged_df['expected_rev'] = merged_df['adr']* merged_df['expected_demand']

In [ ]:
merged_df[(merged_df.dow == 'Friday') & (merged_df.month =='April')].plot(x='adr', y='expected_demand', kind='line')

In [ ]:
merged_df[(merged_df.dow == 'Friday') & (merged_df.month =='April')].plot(x='adr', y='expected_rev', kind='line')

In [ ]:
merged_df

In [ ]:
from scipy.optimize import curve_fit

# Define the demand curve function
def demand_curve(x, a, b, c, d, max_demand):
    demand = a * np.exp(-b * x) + c
    demand = np.where(x <= max_demand, np.minimum(demand, max_demand), demand)
    return demand + d


In [ ]:
x_data = merged_df['adr'].values
y_data = merged_df['expected_demand'].values


initial_guess = [1, 0.01, 1, 1, 100]
bounds = ([0, 0, 0, 0, 0], [np.inf, np.inf, np.inf, np.inf, np.inf])

params, _ = curve_fit(demand_curve, x_data, y_data, bounds=bounds, p0=initial_guess)

a_fit, b_fit, c_fit ,d_fit,max_demand= params

In [ ]:
predicted_demand = demand_curve(x_data, a_fit, b_fit,c_fit,d_fit,max_demand)

In [ ]:
plt.scatter(x_data, y_data, label='Actual Demand')
plt.plot(x_data, predicted_demand, label='Fitted Curve')
plt.xlabel('Price')
plt.ylabel('Demand')
plt.legend()
plt.title('Demand Curve Fit')
plt.show()

In [ ]:
def revenue(price):
    return price * demand_curve(price, a_fit, b_fit,c_fit,d_fit,max_demand)

In [ ]:
objective = lambda price: -revenue(price)
from scipy.optimize import minimize_scalar

result = minimize_scalar(objective, bounds=(60, 180), method='bounded')
optimal_price = result.x
max_revenue = -result.fun
room_sold = demand_curve(optimal_price, a_fit, b_fit,c_fit,d_fit,max_demand)

In [ ]:
print(f"The optimal price to maximize revenue: ${optimal_price}")
print(f"The maximum revenue achievable: ${max_revenue}")
print(f"The expected number of rooms to sell: {room_sold}")

In [ ]:
from scipy.optimize import brentq

def demand_to_price(num_rooms, a, b, c, d, max_demand):
    def root_func(x):
        return num_rooms - (a * np.exp(-b * x) + c)

    try:
        price = brentq(root_func, 0, 200)  # Adjust the interval bounds as needed
    except ValueError:
        # Fallback to default price if no root is found
        price_range=(0, 200)
        price = np.random.uniform(*price_range)

    return price

# from scipy.optimize import brentq

# def demand_to_price(num_rooms, a, b, c, d, max_demand):
   
#     def root_func(x):
#         return num_rooms - (a * np.exp(-b * x) + c)
    
#     # Find the price using numerical root finding
#     price = brentq(root_func, 0, 200)  # Adjust the interval bounds as needed
    
#     return price

In [ ]:
demand_to_price(50,a_fit,b_fit,c_fit,d_fit,max_demand)

In [ ]:
results = pd.DataFrame(columns=['month', 'hotel','room_limit', 'room_type', 'dow', 'optimal_rate', 'expected_rn','expected_rev','optimal_rate_lim_inv'])

In [ ]:
months = merged_df.month.unique()
dow = merged_df.dow.unique()

In [ ]:
merged_df

In [ ]:
#Create a loop to observe if our demand curve fits properly to each demand month and dow


for month in months:
    for day in dow:
        
        print(month,day)
        #get data
        data = merged_df[(merged_df.dow == day) & (merged_df.month ==month)].reset_index()
        
        #remove outlier
        mean = data.adr.mean()
        std_dev = data.adr.std()
       

        # calculate z-scores
        data['z_scores'] = np.abs((data.adr - mean) / std_dev)
        
        #filter out outliers
        data = data[data.z_scores <=2]
        
        ## Fit Demand curve
        x_data = data['adr'].values
        y_data = data['expected_demand'].values
        
        # Try except expression to ensure we get no errors when fitting the demand curve due to our initial guess
        try:
            initial_guess = [1, 0.01, 1, 1,data['total_rns_median'].values[0] ]
            bounds = ([0, 0, 0, 0, 0], [np.inf, np.inf, np.inf, np.inf, np.inf])

        # Fit the demand curve to the data
            params, _ = curve_fit(demand_curve, x_data, y_data, bounds=bounds, p0=initial_guess)
        except:
            if month =='January':
                
                initial_guess = [1, 0.01, 1, 1,40 ]
            else:
                initial_guess = [1, 0.01, 1, 1,50 ]
            bounds = ([0, 0, 0, 0, 0], [np.inf, np.inf, np.inf, np.inf, np.inf])

        # Fit the demand curve to the data
            params, _ = curve_fit(demand_curve, x_data, y_data, bounds=bounds, p0=initial_guess)
        
        # Extract the fitted parameters
        a_fit, b_fit, c_fit ,d_fit,max_demand= params
        
        #visually explore if the demand curve fits the data
        predicted_demand = demand_curve(x_data, a_fit, b_fit,c_fit,d_fit,max_demand)
        
        plt.scatter(x_data, y_data, label='Actual Demand')
        plt.plot(x_data, predicted_demand, label='Fitted Curve')
        plt.xlabel('Price')
        plt.ylabel('Demand')
        plt.legend()
        plt.title('Demand Curve Fit')
        plt.show()

In [ ]:
for hotel in hotels:
    for room_type in room_types:
        for month in months:
            for day in dow:
                # Get data for the specific combination
                data_subset = merged_df[(merged_df['dow'] == day) & 
                                        (merged_df['hotel'] == hotel) & 
                                        (merged_df['room_type'] == room_type) & 
                                        (merged_df['month'] == month)].reset_index()
                
                if data_subset.empty:
                    continue

                # Remove outliers
                mean = data_subset['adr'].mean()
                std_dev = data_subset['adr'].std()
                data_subset['z_scores'] = np.abs((data_subset['adr'] - mean) / std_dev)
                data_subset = data_subset[data_subset['z_scores'] <= 2]

                # Fit demand curve
                x_data = data_subset['adr'].values
                y_data = data_subset['expected_demand'].values

                try:
                    initial_guess = [1, 0.01, 1, 1, data_subset['total_rns_median'].values[0]]
                    bounds = ([0, 0, 0, 0, 0], [np.inf, np.inf, np.inf, np.inf, np.inf])
                    maxfev = 10000  # Increase the number of maximum function evaluations
                    params, _ = curve_fit(demand_curve, x_data, y_data, bounds=bounds, p0=initial_guess, maxfev=maxfev)
                except RuntimeError as e:
                    print(f"Error fitting demand curve for {hotel}, {room_type}, {month}, {day}: {e}")
                    continue

                a_fit, b_fit, c_fit, d_fit, max_demand = params

                # Optimize revenue
                def revenue(price):
                    return price * demand_curve(price, a_fit, b_fit, c_fit, d_fit, max_demand)

                objective = lambda price: -revenue(price)
                optimize = minimize_scalar(objective, bounds=(45, 200), method='bounded')
                optimal_price = optimize.x
                max_revenue = -optimize.fun
                expected_rns = demand_curve(optimal_price, a_fit, b_fit, c_fit, d_fit, max_demand)

                optimal_rate_lim_inv = demand_to_price(data_subset['room_limit'].mean(), a_fit, b_fit, c_fit, d_fit, max_demand)

                new_row = pd.DataFrame({'hotel': hotel,
                                        'room_type': room_type,
                                        'room_limit': data_subset['room_limit'].mean(),
                                        'month': month,
                                        'dow': day,
                                        'optimal_rate': optimal_price,
                                        'expected_rev': max_revenue,
                                        'expected_rn': expected_rns,
                                        'optimal_rate_lim_inv': optimal_rate_lim_inv}, index=[0])
                results = pd.concat([results, new_row], ignore_index=True)


In [ ]:
results

In [ ]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Sort the dataframe by the custom order
results['month'] = pd.Categorical(results['month'], categories=month_order, ordered=True)
results['dow'] = pd.Categorical(results['dow'], categories=dow_order, ordered=True)
results = results.sort_values(['month', 'dow'])

grouped = results.groupby(['month', 'dow'])['optimal_rate'].mean().unstack()

# Create the graph
fig, ax = plt.subplots(figsize=(10, 6))
grouped.plot(ax=ax, kind='bar')
ax.set_xlabel('Day of the Week')
ax.set_ylabel('Optimal Rate')
ax.set_title('Optimal Rate by Month and Day of the Week')

# Customize the appearance (optional)
plt.legend(title='Month', bbox_to_anchor=(1, 1))
plt.xticks(rotation=0)

# Show the graph
plt.show()

In [ ]:
#clean up the results
results['optimal_rate'] = results['optimal_rate'].round()
results['optimal_rate_lim_inv'] = results['optimal_rate_lim_inv'].round()

results['expected_rn'] = results['expected_rn'].round().astype(int)
results['expected_rev'] = results['expected_rev'].round()
results

# Holidays

In [ ]:
holidays =  expanded_df[expanded_df[['new_year', 'easter', 'christmas']].sum(axis=1) != 0]
holidays

In [ ]:
unpivoted = pd.melt(holidays, id_vars=['arrival_date', 'total_rns', 'adr', 'dow', 'month', "hotel", "room_type", 'room_limit'],
                    value_vars=['new_year', 'easter', 'christmas'],
                    var_name='holiday', value_name='holiday_indicator')

In [ ]:
unpivoted[unpivoted.holiday =='christmas']
unpivoted = unpivoted[unpivoted['holiday_indicator'] == 1]
unpivoted

In [ ]:
holiday_adr = unpivoted.groupby(['holiday','adr','holiday_indicator', "hotel", "room_type"]).agg({'room_limit': 'mean', 'total_rns':'sum'}).reset_index()
holiday_rns = unpivoted.groupby(['arrival_date','holiday', "hotel", "room_type"]).agg({'room_limit': 'mean','total_rns':'sum'}).reset_index()
holiday_rns = holiday_rns.groupby(['holiday', "hotel", "room_type"]).agg({'room_limit': 'mean', 'total_rns':['sum','mean','median']}).reset_index()
holiday_rns.columns = ['_'.join(col) for col in holiday_rns.columns]

In [ ]:
holiday_rns

In [ ]:
merged_holidays = pd.merge(holiday_adr, holiday_rns,how='left',left_on=['holiday' ,'hotel', 'room_type'], right_on=['holiday_' ,'hotel_', 'room_type_'],suffixes=('_act', '_tot'))

merged_holidays.drop('holiday_',axis=1,inplace=True)

merged_holidays

In [ ]:
merged_holidays['probability'] = merged_holidays['total_rns']/merged_holidays['total_rns_sum']
merged_holidays['expected_rns'] = merged_holidays['probability'] * merged_holidays['total_rns_median']
merged_holidays = merged_holidays.sort_values(by=['holiday', 'adr'], ascending=[True, False])
merged_holidays['expected_demand']=merged_holidays.groupby(['holiday'])['expected_rns'].cumsum()

In [ ]:
merged_holidays

In [ ]:
unique_holidays = merged_holidays.holiday.unique()


In [ ]:
def demand_to_price(num_rooms, a, b, c, d, max_demand):
    def root_func(x):
        return num_rooms - (a * np.exp(-b * x) + c)

    try:
        price = brentq(root_func, 0, 200)  # Adjust the interval bounds as needed
    except ValueError:
        # Fallback to default price if no root is found
        price_range=(0, 200)
        price = np.random.uniform(*price_range)

    return price

In [ ]:
holiday_results = pd.DataFrame(columns=['holiday', 'optimal_rate', 'expected_rn','expected_rev','optimal_rate_lim_inv'])

for hotel in hotels:
    for room_type in room_types:

        for day in unique_holidays:
                data = merged_holidays[(merged_holidays.holiday == day) & (merged_holidays.hotel == hotel) & (merged_holidays.room_type == room_type) ].reset_index()

                #remove outlier
                mean = data.adr.mean()
                std_dev = data.adr.std()


                # calculate z-scores
                data['z_scores'] = np.abs((data.adr - mean) / std_dev)

                #filter out outliers
                data = data[data.z_scores <=2]

                ## Fit Demand curve
                x_data = data['adr'].values
                y_data = data['expected_demand'].values

                initial_guess = [1, 0.01, 1, 1,data['total_rns_median'].values[0] ]
                bounds = ([0, 0, 0, 0, 0], [np.inf, np.inf, np.inf, np.inf, np.inf])

                try:
                    params, _ = curve_fit(demand_curve, x_data, y_data, bounds=bounds, p0=initial_guess)
                except Exception as e:
                    print(f"Error fitting demand curve for {hotel}, {room_type}: {e}")
                    continue

                # Extract the fitted parameters
                a_fit, b_fit, c_fit ,d_fit,max_demand= params

                a_fit, b_fit, c_fit ,d_fit,max_demand = np.round(a_fit, 3), np.round(b_fit, 3), np.round(c_fit, 3) ,np.round(d_fit, 3),np.round(max_demand, 3)
                #optimze revenue
                objective = lambda price: -revenue(price)



                optimize = minimize_scalar(objective, bounds=(45, 400), method='bounded')
                optimal_price = optimize.x
                max_revenue = -optimize.fun

                expected_rns = demand_curve(optimal_price,a_fit, b_fit, c_fit ,d_fit,max_demand)
                print(50,a_fit,b_fit,c_fit,d_fit,max_demand)
                optimal_rate_lim_inv = demand_to_price(data.room_limit.mean() ,a_fit,b_fit,c_fit,d_fit,max_demand)

                new_row = pd.DataFrame({'holiday':day,
                                        'hotel': hotel,
                                        'room_limit': data.room_limit.mean(),
                               'room_type': room_type,
                               'optimal_rate': optimal_price,
                               'expected_rev':max_revenue,
                               'expected_rn':expected_rns,
                               'optimal_rate_lim_inv':optimal_rate_lim_inv},index=[0])
                holiday_results = pd.concat([holiday_results, new_row], ignore_index=True)

In [ ]:
holiday_results

In [ ]:
results

In [ ]:
room_types

In [ ]:
holiday_results


years = [2020, 2021, 2022, 2023]

holiday_dates = []
for year in years:
    for index, row in holiday_results.iterrows():
        if row['holiday'] == 'christmas':
            date = datetime.date(year, 12, 25)
        elif row['holiday'] == 'easter':
            date = easter(year)
        elif row['holiday'] == 'new_year':
            date = datetime.date(year, 1, 1)

        holiday_dates.append({
            'hotel': row['hotel'],
            'room_type': row['room_type'],
            'month': date.strftime('%B'),
            'dow': date.strftime("%A"),
            'holiday': row['holiday'],
            'optimal_rate': row['optimal_rate'],
            'expected_rn': row['expected_rn'],
            'expected_rev': row['expected_rev'],
            'optimal_rate_lim_inv': row['optimal_rate_lim_inv'],
            'arrival_date': pd.to_datetime(date)
        })

holiday_results_yearly = pd.DataFrame(holiday_dates)




In [ ]:
holiday_results_yearly[holiday_results_yearly.holiday == 'new_year']

In [ ]:
results[(results['month'] == 'April') & (results['room_type'] == 'A')]

In [ ]:
import itertools

hotel_types = ['Resort Hotel', 'City Hotel']
room_types = ['A', 'D', 'E']

combinations = list(itertools.product(hotel_types, room_types))

combinations

combinations_df = pd.DataFrame(combinations, columns=['hotel', 'room_type'])
combinations_df

In [ ]:
month_dict = {month: index for index, month in enumerate(pd.date_range('2020-01-01', periods=12, freq='M').strftime('%B'), 1)}


new_data = pd.DataFrame()

for year in range(2020, 2024):
    for month in month_dict.values():
        start_date = pd.to_datetime(f'{year}-{month}-01').replace(day=1)
        end_date = pd.to_datetime(f'{year}-{month}-01').replace(day=1) + pd.offsets.MonthEnd(0)
        date_range = pd.date_range(start_date, end_date, freq='D')
        df = pd.DataFrame(date_range, columns=['arrival_date'])
        df['dow'] = df['arrival_date'].dt.day_name()
        df['month'] = df['arrival_date'].dt.month_name()

        result_df = df.assign(key=1).merge(combinations_df.assign(key=1), on='key').drop('key', axis=1)
        new_data = pd.concat([new_data, result_df], ignore_index=True)
new_data

In [ ]:
final_data = pd.merge(new_data, results, how='left', on=['dow', 'hotel', 'room_type', 'month'])
final_data

In [ ]:
holiday_results_yearly['arrival_date'] = pd.to_datetime(holiday_results_yearly['arrival_date'])
final_data['arrival_date'] = pd.to_datetime(final_data['arrival_date'])

In [ ]:
merged_df = pd.merge(final_data, holiday_results_yearly[['arrival_date', 'hotel', 'room_type', 'month', 'dow', 'optimal_rate', 'expected_rn', 'expected_rev', 'optimal_rate_lim_inv']], how='left', on=['arrival_date', 'hotel','room_type','month'], suffixes=('', '_holiday'))
merged_df

In [ ]:
merged_df[merged_df.arrival_date == '2020-01-01']

In [ ]:
final_data['optimal_rate'] = merged_df['optimal_rate_holiday'].combine_first(final_data['optimal_rate'])
final_data['expected_rn'] = merged_df['expected_rn_holiday'].combine_first(final_data['expected_rn'])
final_data['expected_rev'] = merged_df['expected_rev_holiday'].combine_first(final_data['expected_rev'])
final_data['optimal_rate_lim_inv'] = merged_df['optimal_rate_lim_inv_holiday'].combine_first(final_data['optimal_rate_lim_inv'])


In [ ]:
final_data.isna().sum()

In [ ]:
final_data

In [ ]:
data_backup

In [ ]:
data_backup['arrival_date_transformed'] = pd.to_datetime(data_backup['arrival_date_transformed'])
final_data['arrival_date'] = pd.to_datetime(final_data['arrival_date'])

In [ ]:
rev_opt_booking = pd.merge(data_backup, final_data, right_on=['arrival_date', 'hotel', 'room_type'], left_on=['arrival_date_transformed', 'hotel', 'reserved_room_type'], how='left')
rev_opt_booking.to_csv('rev_opt_booking.csv', index=False)

In [ ]:
rev_opt_booking.columns = map(lambda x: str(x).upper(), rev_opt_booking.columns)


In [ ]:
rev_opt_booking["ARRIVAL_DATE"] = pd.to_datetime(rev_opt_booking["ARRIVAL_DATE"]).dt.strftime("%Y-%m-%d %H:%M:%S.%f")
rev_opt_booking["ARRIVAL_DATE_TRANSFORMED"] = pd.to_datetime(rev_opt_booking["ARRIVAL_DATE_TRANSFORMED"]).dt.strftime("%Y-%m-%d %H:%M:%S.%f")

In [ ]:
df_model=session.createDataFrame(
        rev_opt_booking.values.tolist(),
        schema=rev_opt_booking.columns.tolist())
df_model.write.mode("overwrite").save_as_table("TTH_DB.TTH_REV_OPT_Schema.REV_OPT_BOOKING_CLEANED")

In [ ]:
rev_opt_booking.info()